# Computer Vision - UOCIS-510 - Homework 1
__Part 1: (4/15 pts)__

Implement Nearest Neighbor classifier for CIFAR10 dataset.

Report the accuracy on test set for both L1 and L2 distance.

In [4]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler as SRS
import torchvision
from torchvision import transforms
import numpy as np

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print('Using device:', device)

In [5]:
# parameters
k = 1 # Amount of nearest neighbors
bs = 1 # Batch Size (DON'T CHANGE FROM 1)
numworkers = 1 # number of workers
testPercent = 0.1 # % of dataset to use

In [232]:
transform = transforms.Compose([
                       transforms.ToTensor(),
])
cifar10 = torchvision.datasets.CIFAR10("../datasets",train=True,download=True, transform=transform)

indices = np.arange(len(cifar10))
np.random.shuffle(indices)
size = int(testPercent*len(cifar10))
ind = indices[:size]
print(f"Using {size} images")

sampler = SRS(ind)

dataSet = DataLoader(cifar10,
                     batch_size=bs,
                     sampler=sampler,
                     shuffle=False,
                     num_workers=numworkers,
                     )


Files already downloaded and verified
Using 5000 images


In [244]:
def LPDist(image, dataset, k=1, lp=2):
    '''
    Computes the Lp loss between a dataset
    '''
    #dist = torch.tensor(len(dataset))
    dist = torch.FloatTensor(
        [torch.pow(
            torch.sum(
                torch.pow(
                    torch.sub(image,dataset[i][0]), # sub
                    lp)# pow
                    ),# sum
            1/lp) # pow 
         for i in range(len(dataset))])
    #return dist, torch.argmin(dist)
    return dist, torch.argsort(dist)[1:k+1] # The closest is the input itself

In [245]:
def Closest(mins, dataset):
    '''
    Uses a single pass by storing in a hashtable instead of looping twice
    '''
    dict = {}
    for m in mins:
        k = dataset[m][1]
        if k in dict.keys():
            dict[k] += 1
        else:
            dict[k] = 1
    #print(dict)
    max = torch.zeros(1)
    for key in dict.keys():
        if dict[key] > max:
            max[0] = key
    return max
        

In [246]:
def getAccuracy(true, label):
    #print(true)
    #print(label)
    if true.item() == label: return 1
    return 0

In [247]:
correct_lp1 = 0
correct_lp2 = 0
total = 0
for idx, (img, label) in enumerate(dataSet):
    print(f"Step {idx}/{len(dataSet)}")
    #print(f"Correct Label: {label}")
    dist_lp1, min_lp1 = LPDist(img,cifar10,k=k,lp=1)
    dist_lp2, min_lp2 = LPDist(img,cifar10,k=k,lp=2)
    label_lp1 = Closest(min_lp1, cifar10)
    #print(f"LP1 label = {label_lp1}")
    label_lp2 = Closest(min_lp2, cifar10)
    #print(f"LP2 label = {label_lp2}")
    correct_lp1 += getAccuracy(label, label_lp1)
    correct_lp2 += getAccuracy(label, label_lp2)
    total += bs
    print(f"LP1 Accuracy: {correct_lp1/total}")
    print(f"LP2 Accuracy: {correct_lp2/total}")

print("Complete")
print(f"LP1 Accuracy: {correct_lp1/total}")
print(f"LP2 Accuracy: {correct_lp2/total}")
   


Step 0/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 1.0
Step 1/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.5
Step 2/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.3333333333333333
Step 3/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.25
Step 4/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.2
Step 5/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.3333333333333333
Step 6/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.2857142857142857
Step 7/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.25
Step 8/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.3333333333333333
Step 9/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.3
Step 10/5000
LP1 Accuracy: 0.0
LP2 Accuracy: 0.2727272727272727
Step 11/5000
LP1 Accuracy: 0.08333333333333333
LP2 Accuracy: 0.3333333333333333
Step 12/5000
LP1 Accuracy: 0.07692307692307693
LP2 Accuracy: 0.38461538461538464
Step 13/5000
LP1 Accuracy: 0.07142857142857142
LP2 Accuracy: 0.42857142857142855
Step 14/5000
LP1 Accuracy: 0.06666666666666667
LP2 Accuracy: 0.4666666666666667
Step 15/5000
LP1 Accuracy: 0.0625
LP2 Accuracy: 0.4375
Step 16/5000
LP1 

LP1 Accuracy: 0.10810810810810811
LP2 Accuracy: 0.34234234234234234
Step 111/5000
LP1 Accuracy: 0.10714285714285714
LP2 Accuracy: 0.3392857142857143
Step 112/5000
LP1 Accuracy: 0.10619469026548672
LP2 Accuracy: 0.336283185840708
Step 113/5000
LP1 Accuracy: 0.11403508771929824
LP2 Accuracy: 0.3333333333333333
Step 114/5000
LP1 Accuracy: 0.11304347826086956
LP2 Accuracy: 0.33043478260869563
Step 115/5000
LP1 Accuracy: 0.11206896551724138
LP2 Accuracy: 0.3275862068965517
Step 116/5000
LP1 Accuracy: 0.1111111111111111
LP2 Accuracy: 0.3247863247863248
Step 117/5000
LP1 Accuracy: 0.11016949152542373
LP2 Accuracy: 0.3305084745762712
Step 118/5000
LP1 Accuracy: 0.1092436974789916
LP2 Accuracy: 0.33613445378151263
Step 119/5000
LP1 Accuracy: 0.10833333333333334
LP2 Accuracy: 0.3333333333333333
Step 120/5000
LP1 Accuracy: 0.10743801652892562
LP2 Accuracy: 0.3305785123966942
Step 121/5000
LP1 Accuracy: 0.10655737704918032
LP2 Accuracy: 0.32786885245901637
Step 122/5000
LP1 Accuracy: 0.10569105691

LP1 Accuracy: 0.08837209302325581
LP2 Accuracy: 0.31627906976744186
Step 215/5000
LP1 Accuracy: 0.08796296296296297
LP2 Accuracy: 0.3148148148148148
Step 216/5000
LP1 Accuracy: 0.08755760368663594
LP2 Accuracy: 0.31797235023041476
Step 217/5000
LP1 Accuracy: 0.0871559633027523
LP2 Accuracy: 0.3165137614678899
Step 218/5000
LP1 Accuracy: 0.0867579908675799
LP2 Accuracy: 0.3150684931506849
Step 219/5000
LP1 Accuracy: 0.08636363636363636
LP2 Accuracy: 0.3181818181818182
Step 220/5000
LP1 Accuracy: 0.08597285067873303
LP2 Accuracy: 0.3167420814479638
Step 221/5000
LP1 Accuracy: 0.08558558558558559
LP2 Accuracy: 0.31981981981981983
Step 222/5000
LP1 Accuracy: 0.08520179372197309
LP2 Accuracy: 0.3183856502242152
Step 223/5000
LP1 Accuracy: 0.08482142857142858
LP2 Accuracy: 0.32142857142857145
Step 224/5000
LP1 Accuracy: 0.08444444444444445
LP2 Accuracy: 0.32
Step 225/5000
LP1 Accuracy: 0.084070796460177
LP2 Accuracy: 0.3185840707964602
Step 226/5000
LP1 Accuracy: 0.08370044052863436
LP2 Accu

LP1 Accuracy: 0.09119496855345911
LP2 Accuracy: 0.31761006289308175
Step 318/5000
LP1 Accuracy: 0.09090909090909091
LP2 Accuracy: 0.31974921630094044
Step 319/5000
LP1 Accuracy: 0.09375
LP2 Accuracy: 0.321875
Step 320/5000
LP1 Accuracy: 0.09345794392523364
LP2 Accuracy: 0.32087227414330216
Step 321/5000
LP1 Accuracy: 0.09316770186335403
LP2 Accuracy: 0.3198757763975155
Step 322/5000
LP1 Accuracy: 0.09287925696594428
LP2 Accuracy: 0.3188854489164087
Step 323/5000
LP1 Accuracy: 0.09259259259259259
LP2 Accuracy: 0.32098765432098764
Step 324/5000
LP1 Accuracy: 0.09230769230769231
LP2 Accuracy: 0.3230769230769231
Step 325/5000
LP1 Accuracy: 0.09202453987730061
LP2 Accuracy: 0.3220858895705521
Step 326/5000
LP1 Accuracy: 0.09174311926605505
LP2 Accuracy: 0.3211009174311927
Step 327/5000
LP1 Accuracy: 0.09146341463414634
LP2 Accuracy: 0.3231707317073171
Step 328/5000
LP1 Accuracy: 0.0911854103343465
LP2 Accuracy: 0.3252279635258359
Step 329/5000
LP1 Accuracy: 0.09090909090909091
LP2 Accuracy:

LP1 Accuracy: 0.08333333333333333
LP2 Accuracy: 0.3333333333333333
Step 420/5000
LP1 Accuracy: 0.0831353919239905
LP2 Accuracy: 0.332541567695962
Step 421/5000
LP1 Accuracy: 0.08293838862559241
LP2 Accuracy: 0.3341232227488152
Step 422/5000
LP1 Accuracy: 0.08274231678486997
LP2 Accuracy: 0.3333333333333333
Step 423/5000
LP1 Accuracy: 0.08254716981132075
LP2 Accuracy: 0.33254716981132076
Step 424/5000
LP1 Accuracy: 0.08235294117647059
LP2 Accuracy: 0.33176470588235296
Step 425/5000
LP1 Accuracy: 0.08215962441314555
LP2 Accuracy: 0.33098591549295775
Step 426/5000
LP1 Accuracy: 0.08196721311475409
LP2 Accuracy: 0.3325526932084309
Step 427/5000
LP1 Accuracy: 0.08177570093457943
LP2 Accuracy: 0.3341121495327103
Step 428/5000
LP1 Accuracy: 0.08158508158508158
LP2 Accuracy: 0.3333333333333333
Step 429/5000
LP1 Accuracy: 0.08139534883720931
LP2 Accuracy: 0.3325581395348837
Step 430/5000
LP1 Accuracy: 0.08120649651972157
LP2 Accuracy: 0.33178654292343385
Step 431/5000
LP1 Accuracy: 0.0810185185

KeyboardInterrupt: 

# Results

LP1 Accuracy: 0.089

LP2 Accuracy: 0.3320

Note: This takes a long time to run. Stopped where I did because my computer can't crunch very fast and it appeared the results were converging anyways.

# Part 2: (8/15 pts)

Implement a Convolutional Neural Network for CIFAR-10 dataset.

You can follow this
[tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py)
if need: 

Implement different network architectures and report your results. (for optimization, you can just following the tutorial, but feel free to experiment with different ones)

In [265]:
# parameters
epochs = 20
bs = 10 # Batch Size 
numworkers = 5 # number of workers
testPercent = 1 # % of dataset to use
transform = transforms.Compose([
                       transforms.ToTensor(),
])
cifar10 = torchvision.datasets.CIFAR10("../datasets",train=True,download=True, transform=transform)

indices = np.arange(len(cifar10))
np.random.shuffle(indices)
size = int(testPercent*len(cifar10))
ind = indices[:size]
print(f"Using {size} images")

sampler = SRS(ind)

dataSet = DataLoader(cifar10,
                     batch_size=bs,
                     sampler=sampler,
                     shuffle=False,
                     num_workers=numworkers,
                     )
cifar10Test = torchvision.datasets.CIFAR10("../datasets",train=False,download=True, transform=transform)
testSet = DataLoader(cifar10Test,
                      batch_size=bs,
                      shuffle=True,
                      num_workers=numworkers,
                      )


Files already downloaded and verified
Using 50000 images
Files already downloaded and verified


In [266]:
def convBlock(_in,_out,kernel_size=3,
              stride=1,
              padding=1,
              inplace=True,
              dropout=False,
              p=0.5,
             ):
    
    layers = [nn.Conv2d(_in,_out,kernel_size=kernel_size,stride=stride,padding=padding)]
    layers.append(nn.ReLU(inplace=inplace))
    if dropout:
        layers.append(nn.Dropout(p=p,inplace=inplace))
        
    return layers
    

In [267]:
class ExampleNet(nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.reg = nn.ReLU()
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(self.reg(self.conv1(x)))
        x = self.pool(self.reg(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.reg(self.fc1(x))
        x = self.reg(self.fc2(x))
        x = self.fc3(x)
        return x

In [268]:
import torch.optim as optim
net = ExampleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataSet, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            #running_loss = 0.0
    print(f"Epoch Loss: {running_loss/(i+1)}")
print('Finished Training')


[1,  1000] loss: 1.150
[1,  2000] loss: 2.253
[1,  3000] loss: 3.257
[1,  4000] loss: 4.187
[1,  5000] loss: 5.053
Epoch Loss: 2.0210932716369627
[2,  1000] loss: 0.827
[2,  2000] loss: 1.628
[2,  3000] loss: 2.402
[2,  4000] loss: 3.152
[2,  5000] loss: 3.900
Epoch Loss: 1.5600294486641884
[3,  1000] loss: 0.723
[3,  2000] loss: 1.444
[3,  3000] loss: 2.152
[3,  4000] loss: 2.847
[3,  5000] loss: 3.529
Epoch Loss: 1.4116766491293906
[4,  1000] loss: 0.672
[4,  2000] loss: 1.345
[4,  3000] loss: 2.001
[4,  4000] loss: 2.659
[4,  5000] loss: 3.316
Epoch Loss: 1.3262041688799857
[5,  1000] loss: 0.637
[5,  2000] loss: 1.264
[5,  3000] loss: 1.889
[5,  4000] loss: 2.513
[5,  5000] loss: 3.137
Epoch Loss: 1.254721093505621
[6,  1000] loss: 0.600
[6,  2000] loss: 1.202
[6,  3000] loss: 1.787
[6,  4000] loss: 2.390
[6,  5000] loss: 2.982
Epoch Loss: 1.192643980705738
[7,  1000] loss: 0.565
[7,  2000] loss: 1.133
[7,  3000] loss: 1.698
[7,  4000] loss: 2.275
[7,  5000] loss: 2.849
Epoch Loss:

In [269]:
net.eval()
running_loss = 0
for i, (img, label) in enumerate(testSet):
    pred = net(img)
    loss = criterion(pred,label)
    running_loss += loss.item()
print(f"Test Loss: {running_loss/(i+1)}")

Test Loss: 1.1992270113900303


Test Loss: 1.1992270113900303

In [270]:
class shallowNetwork(nn.Module):
    def __init__(self):
        super(shallowNetwork,self).__init__()
        convLayers = []
        for i in range(3):
            convLayers += convBlock(3,3)
        self.convs= nn.Sequential(*convLayers)
        self.size = 3*32*32 # Size of single image
        self.linear = nn.Linear(self.size, 10)
        
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self.size)
        x = self.linear(x)
        return x

In [271]:
net = shallowNetwork()
net.train()
optimizer = optim.AdamW(net.parameters(), lr=0.01)
for epoch in range(epochs):
    running_loss = 0
    for i, (data,label) in enumerate(dataSet):
        optimizer.zero_grad()
        pred = net(data)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
    print(f"Epoch {epoch} Loss: {running_loss/(i+1)}")

[1,  1000] loss: 1.155
[1,  2000] loss: 2.307
[1,  3000] loss: 3.460
[1,  4000] loss: 4.612
[1,  5000] loss: 5.765
Epoch 0 Loss: 2.3060174372673035
[2,  1000] loss: 1.152
[2,  2000] loss: 2.304
[2,  3000] loss: 3.457
[2,  4000] loss: 4.610
[2,  5000] loss: 5.762
Epoch 1 Loss: 2.3048549178600313
[3,  1000] loss: 1.153
[3,  2000] loss: 2.305
[3,  3000] loss: 3.458
[3,  4000] loss: 4.610
[3,  5000] loss: 5.762
Epoch 2 Loss: 2.3047721650123596
[4,  1000] loss: 1.153
[4,  2000] loss: 2.305
[4,  3000] loss: 3.458
[4,  4000] loss: 4.611
[4,  5000] loss: 5.763
Epoch 3 Loss: 2.305278517484665
[5,  1000] loss: 1.152
[5,  2000] loss: 2.305
[5,  3000] loss: 3.458
[5,  4000] loss: 4.610
[5,  5000] loss: 5.763
Epoch 4 Loss: 2.305134665584564
[6,  1000] loss: 1.152
[6,  2000] loss: 2.305
[6,  3000] loss: 3.457
[6,  4000] loss: 4.609
[6,  5000] loss: 5.761
Epoch 5 Loss: 2.304594834518433
[7,  1000] loss: 1.153
[7,  2000] loss: 2.305
[7,  3000] loss: 3.458
[7,  4000] loss: 4.610
[7,  5000] loss: 5.763


In [272]:
net.eval()
running_loss = 0
for i, (img, label) in enumerate(testSet):
    pred = net(img)
    loss = criterion(pred,label)
    running_loss += loss.item()
print(f"Test Loss: {running_loss/(i+1)}")

Test Loss: 2.306026217699051


Test Loss: 2.306026217699051

In [273]:
class DeeperNetwork(nn.Module):
    def __init__(self,numLayers=10):
        super(DeeperNetwork,self).__init__()
        convLayers = []
        for i in range(numLayers):
            convLayers += convBlock(3,3)
        self.convs= nn.Sequential(*convLayers)
        self.size = 3*32*32 # Size of single image
        self.linear = nn.Sequential(nn.Linear(self.size,10),nn.ReLU())
        
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self.size)
        x = self.linear(x)
        return x

In [274]:
net = DeeperNetwork()
net.train()
optimizer = optim.AdamW(net.parameters(), lr=0.01)
for epoch in range(epochs):
    running_loss = 0
    for i, (data,label) in enumerate(dataSet):
        optimizer.zero_grad()
        pred = net(data)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
    print(f"Epoch {epoch} Loss: {running_loss/(i+1)}")

[1,  1000] loss: 1.152
[1,  2000] loss: 2.304
[1,  3000] loss: 3.455
[1,  4000] loss: 4.606
[1,  5000] loss: 5.758
Epoch 0 Loss: 2.303054624128342
[2,  1000] loss: 1.151
[2,  2000] loss: 2.303
[2,  3000] loss: 3.454
[2,  4000] loss: 4.605
[2,  5000] loss: 5.756
Epoch 1 Loss: 2.3025853633880615
[3,  1000] loss: 1.151
[3,  2000] loss: 2.303
[3,  3000] loss: 3.454
[3,  4000] loss: 4.605
[3,  5000] loss: 5.756
Epoch 2 Loss: 2.3025853633880615
[4,  1000] loss: 1.151
[4,  2000] loss: 2.303
[4,  3000] loss: 3.454
[4,  4000] loss: 4.605
[4,  5000] loss: 5.756
Epoch 3 Loss: 2.3025853633880615
[5,  1000] loss: 1.151
[5,  2000] loss: 2.303
[5,  3000] loss: 3.454
[5,  4000] loss: 4.605
[5,  5000] loss: 5.756
Epoch 4 Loss: 2.3025853633880615
[6,  1000] loss: 1.151
[6,  2000] loss: 2.303
[6,  3000] loss: 3.454
[6,  4000] loss: 4.605
[6,  5000] loss: 5.756
Epoch 5 Loss: 2.3025853633880615
[7,  1000] loss: 1.151
[7,  2000] loss: 2.303
[7,  3000] loss: 3.454
[7,  4000] loss: 4.605
[7,  5000] loss: 5.75

In [275]:
net.eval()
running_loss = 0
for i, (img, label) in enumerate(testSet):
    pred = net(img)
    loss = criterion(pred,label)
    running_loss += loss.item()
print(f"Test Loss: {running_loss/(i+1)}")

Test Loss: 2.3025853633880615


Test Loss: 2.3025853633880615

# Part 3: (3/15 pts)

Implement Linear Classifier for CIFAR-10 dataset based on code from Part 2. (i.e. treat it as neural networks but use linear layers)

Report the accuracy on test set for both with and without bias term:

i.e. $y = Wx$ or $y = Wx + b$

And both single linear layer and multiple linear layers.

Report and compare results.

In [276]:
class SingleLinearNetwork(nn.Module):
    def __init__(self,numLayers=10):
        super(SingleLinearNetwork,self).__init__()
        convLayers = []
        for i in range(numLayers):
            convLayers += convBlock(3,3)
        self.convs= nn.Sequential(*convLayers)
        self.size = 3*32*32 # Size of single image
        self.linear = nn.Sequential(nn.Linear(self.size,10),
                                    nn.ReLU(),
                                   )
        
    def forward(self,x):
        x = x.view(-1,self.size)
        x = self.linear(x)
        return x

In [277]:
net = SingleLinearNetwork()
optimizer = optim.AdamW(net.parameters(), lr=0.01)
for epoch in range(epochs):
    running_loss = 0
    for i, (data,label) in enumerate(dataSet):
        optimizer.zero_grad()
        pred = net(data)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
    print(f"Epoch {epoch} Loss: {running_loss/(i+1)}")

[1,  1000] loss: 1.172
[1,  2000] loss: 2.324
[1,  3000] loss: 3.475
[1,  4000] loss: 4.626
[1,  5000] loss: 5.778
Epoch 0 Loss: 2.3110009608745576
[2,  1000] loss: 1.151
[2,  2000] loss: 2.303
[2,  3000] loss: 3.454
[2,  4000] loss: 4.605
[2,  5000] loss: 5.756
Epoch 1 Loss: 2.3025853633880615
[3,  1000] loss: 1.151
[3,  2000] loss: 2.303
[3,  3000] loss: 3.454
[3,  4000] loss: 4.605
[3,  5000] loss: 5.756
Epoch 2 Loss: 2.3025853633880615
[4,  1000] loss: 1.151
[4,  2000] loss: 2.303
[4,  3000] loss: 3.454
[4,  4000] loss: 4.605
[4,  5000] loss: 5.756
Epoch 3 Loss: 2.3025853633880615
[5,  1000] loss: 1.151
[5,  2000] loss: 2.303
[5,  3000] loss: 3.454
[5,  4000] loss: 4.605
[5,  5000] loss: 5.756
Epoch 4 Loss: 2.3025853633880615
[6,  1000] loss: 1.151
[6,  2000] loss: 2.303
[6,  3000] loss: 3.454
[6,  4000] loss: 4.605
[6,  5000] loss: 5.756
Epoch 5 Loss: 2.3025853633880615
[7,  1000] loss: 1.151
[7,  2000] loss: 2.303
[7,  3000] loss: 3.454
[7,  4000] loss: 4.605
[7,  5000] loss: 5.7

In [278]:
net.eval()
running_loss = 0
for i, (img, label) in enumerate(testSet):
    pred = net(img)
    loss = criterion(pred,label)
    running_loss += loss.item()
print(f"Test Loss: {running_loss/(i+1)}")

Test Loss: 2.3025853633880615


Test Loss: 2.3025853633880615

In [279]:
class LinearNetwork(nn.Module):
    def __init__(self,numLayers=10):
        super(LinearNetwork,self).__init__()
        convLayers = []
        for i in range(numLayers):
            convLayers += convBlock(3,3)
        self.convs= nn.Sequential(*convLayers)
        self.size = 3*32*32 # Size of single image
        self.linear = nn.Sequential(nn.Linear(self.size,1000),
                                    nn.ReLU(),
                                    nn.Linear(1000,500),
                                    nn.ReLU(),
                                    nn.Linear(500,10),
                                   )
        
    def forward(self,x):
        x = x.view(-1,self.size)
        x = self.linear(x)
        return x

In [280]:
net = LinearNetwork()
optimizer = optim.AdamW(net.parameters(), lr=0.01)
for epoch in range(epochs):
    running_loss = 0
    for i, (data,label) in enumerate(dataSet):
        optimizer.zero_grad()
        pred = net(data)
        loss = criterion(pred, label)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
    print(f"Epoch {epoch} Loss: {running_loss/(i+1)}")

[1,  1000] loss: 1.243
[1,  2000] loss: 2.395
[1,  3000] loss: 3.547
[1,  4000] loss: 4.700
[1,  5000] loss: 5.852
Epoch 0 Loss: 2.340956567239761
[2,  1000] loss: 1.153
[2,  2000] loss: 2.305
[2,  3000] loss: 3.458
[2,  4000] loss: 4.610
[2,  5000] loss: 5.763
Epoch 1 Loss: 2.3051849474430086
[3,  1000] loss: 1.153
[3,  2000] loss: 2.305
[3,  3000] loss: 3.458
[3,  4000] loss: 4.610
[3,  5000] loss: 5.763
Epoch 2 Loss: 2.3051904609680176
[4,  1000] loss: 1.152
[4,  2000] loss: 2.305
[4,  3000] loss: 3.458
[4,  4000] loss: 4.611
[4,  5000] loss: 5.763
Epoch 3 Loss: 2.3053984189987182
[5,  1000] loss: 1.153
[5,  2000] loss: 2.305
[5,  3000] loss: 3.458
[5,  4000] loss: 4.610
[5,  5000] loss: 5.763
Epoch 4 Loss: 2.305188440847397
[6,  1000] loss: 1.152
[6,  2000] loss: 2.305
[6,  3000] loss: 3.458
[6,  4000] loss: 4.610
[6,  5000] loss: 5.762
Epoch 5 Loss: 2.3049027447223662
[7,  1000] loss: 1.152
[7,  2000] loss: 2.305
[7,  3000] loss: 3.457
[7,  4000] loss: 4.610
[7,  5000] loss: 5.763

In [281]:
net.eval()
running_loss = 0
for i, (img, label) in enumerate(testSet):
    pred = net(img)
    loss = criterion(pred,label)
    running_loss += loss.item()
print(f"Test Loss: {running_loss/(i+1)}")

Test Loss: 2.3046057651042937


Test Loss: 2.3046057651042937

# Conclusion

Deeper doesn't really mean better. There's more going into networks that make them better besides depth. Things like pooling layers, batch norms, and dropout make a big difference.


I ran short epochs because my home computer can't handle much computation. 

# Extra Credits: (5 pts, pick either one)

(a) Fine-tune a ResNet-50 model pre-trained on ImageNet for classifying the Caltech-UCSD Birds dataset (Links to an external site.). 

Post (Links to an external site.) on how to fine-tune models in PyTorch.

In [12]:
import os
from torchvision import models
from torchvision import datasets

resnet50 = models.resnet50(pretrained=True)

In [15]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = "../datasets/birds/200.Common_Yellowthroat"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/birds/200\\.Common_Yellowthroat/train'